# Función para poder utilizar la red neuronal para nuevos datos.

Es necesario escalar y transformar los datos para que sean compatibles con la red neuronal es importante entonces utilizar el siguiente codigo y que los nuevos datos sigan la misma distribución de datos usados en la base de datos, sino generara errores y distorciones en la predicción

In [ ]:
import joblib
import pandas as pd
import numpy as np

# Función para transformar una nueva fila o DataFrame de datos
def preparar_nuevos_datos(df_nuevo: pd.DataFrame) -> pd.DataFrame:
    """
    Recibe un DataFrame con al menos las mismas columnas que los datos originales
    (ejemplo: 'home_ownership', 'purpose', 'earliest_cr_line', etc.).
    Devuelve un DataFrame transformado y escalado, listo para ser ingresado al modelo.
    """
    scaler = joblib.load("scaler.joblib")
    model_columns = joblib.load("model_columns.joblib")
    # Copiamos para no alterar el DataFrame original
    df_nuevo = df_nuevo.copy()

    # 1. Ajustar la columna 'home_ownership'
    #   (Si existía la eliminación de valores no deseados, aquí simplemente los forzamos a un valor común)
    df_nuevo.loc[df_nuevo['home_ownership'].isin(['OTHER', 'NONE', 'ANY']), 'home_ownership'] = 'OTHER'

    # 2. Unificar varias categorías de 'purpose' en 'other'
    purpose_columns = [
        "moving", "vacation", "house", "educational", "renewable_energy",
        "wedding", "medical", "car", "small_business", "major_purchase", "other"
    ]
    df_nuevo.loc[df_nuevo["purpose"].isin(purpose_columns), "purpose"] = "other"

    # 3. Manejo de la columna 'loan_status' (si existiera en el nuevo set, la ignoramos o la usamos)
    #    Aquí, asumimos que la columna 'loan_status' ya no es necesaria para la predicción.
    #    Igualar categorías con 'Charged Off' o 'Fully Paid' si fuera requerido, o eliminar la columna.
    if 'loan_status' in df_nuevo.columns:
        df_nuevo.drop('loan_status', axis=1, inplace=True, errors='ignore')

    # 4. Creación de campo 'target' en caso de que se requiera, pero si es para predecir, normalmente no se utiliza.
    #    df_nuevo['target'] = ...

    # 5. Transformar 'earliest_cr_line' en año y mes
    if 'earliest_cr_line' in df_nuevo.columns:
        df_nuevo['earliest_cr_line'] = pd.to_datetime(df_nuevo['earliest_cr_line'], format='%b-%Y', errors='coerce')
        df_nuevo['earliest_cr_year'] = df_nuevo['earliest_cr_line'].dt.year
        df_nuevo['earliest_cr_month'] = df_nuevo['earliest_cr_line'].dt.month
        df_nuevo.drop('earliest_cr_line', axis=1, inplace=True)

    # 6. One-hot encoding para columnas tipo string
    string_columns = df_nuevo.select_dtypes(include=['object']).columns.tolist()
    df_nuevo = pd.get_dummies(df_nuevo, columns=string_columns, drop_first=False)

    # 7. Asegurar que las columnas coincidan con las del modelo
    #    (Por ejemplo, usar las mismas columnas que dEntrenamiento o dEntrenamiento_balanced)
    #    Suponemos que 'columnas_modelo' es la lista exacta de columnas que espera el modelo.
    columnas_modelo = [
        col for col in model_columns if col != 'target'
    ]
    # Añadir columnas faltantes con valor 0
    for col in columnas_modelo:
        if col not in df_nuevo.columns:
            df_nuevo[col] = 0
    # Quitar columnas sobrantes que el modelo no use
    df_nuevo = df_nuevo[columnas_modelo]

    # 8. Aplicar el scaler (StandardScaler) ya entrenado
    #    Queda en forma de array, pero lo devolvemos como DataFrame con las mismas columnas
    data_escalada = scaler.transform(df_nuevo)
    df_nuevo_escalado = pd.DataFrame(data_escalada, columns=columnas_modelo)

    # Retornamos el DataFrame escalado con las columnas ajustadas
    return df_nuevo_escalado

## Ejemplo de uso

Un ejemplo de como usar la función importando el scaler y la red neuronal para nuevas predicciones

In [2]:
import pandas as pd
df = pd.read_csv("Base_datos_Entrenamiento.csv")
from tensorflow.keras.models import load_model
import joblib
joblib.load("scaler.joblib")
modelo_cargado=load_model("best_model.h5")
probar=df[0:4]
probar=preparar_nuevos_datos(probar)

prob_pred = modelo_cargado.predict(probar)
print("Probabilidad predicción:", prob_pred)

2025-01-22 01:41:58.937628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-22 01:41:58.995048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-22 01:41:59.021624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 01:41:59.216525: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-22 01:42:00.721103: W tensorflow/compiler/tf2

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step
Probabilidad predicción: [[0.22547604]
 [0.2472183 ]
 [0.17304985]
 [0.17074163]]


I0000 00:00:1737528125.105821  127834 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
